In [1]:
import os
%pwd
os.chdir("../")
%pwd

'c:\\Users\\Neha Singhal\\OneDrive\\Desktop\\NLP Projects\\TextSummarizer'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    data_path:Path
    model_chkpt : Path
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    per_device_eval_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int

In [3]:
from TextSummarizer.constant import *
from TextSummarizer.utils.common import read_yaml,create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(

            root_dir = config.root_dir,
            data_path = config.data_path,
            model_chkpt  =  config.model_chkpt,
            num_train_epochs  = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps

        )
        return model_trainer_config



In [5]:
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch


c:\myfolder\envs\textsum\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/70/f9/c381bcdd0c3829d723aa14eec8e75c6c377b4ca61ec68b8093d9f35fc7a7/accelerate-0.21.0-py3-none-any.whl.metadata
  Using cached accelerate-0.21.0-py3-none-any.whl.metadata (17 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)


In [7]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_chkpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_chkpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)
        print(self.config)
        trainer_args = TrainingArguments(
        output_dir = self.config.root_dir,
        num_train_epochs=self.config.num_train_epochs,
        warmup_steps=self.config.warmup_steps,
        per_device_train_batch_size=self.config.per_device_train_batch_size,
        per_device_eval_batch_size=self.config.per_device_eval_batch_size,
        weight_decay=self.config.weight_decay,
        logging_steps=self.config.logging_steps,
        evaluation_strategy=self.config.evaluation_strategy,
        eval_steps=self.config.eval_steps,
        save_steps=self.config.save_steps,
        gradient_accumulation_steps=self.config.gradient_accumulation_steps
           )
        
        trainer = Trainer(model=model_pegasus,args=trainer_args,
                  tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['test'],
                  eval_dataset=dataset_samsum_pt['validation'])
        
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [8]:
!pip install accelerate -U

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config
                                    )
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-08-12 13:31:10,169:INFO:common:yaml file : config\config.yaml loaded successfully] 
[2023-08-12 13:31:10,175:INFO:common:yaml file : params.yaml loaded successfully] 
[2023-08-12 13:31:10,177:INFO:common:created directory at :artifacts] 
[2023-08-12 13:31:10,178:INFO:common:created directory at :artifacts/model_trainer] 


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\myfolder\envs\textsum\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ModelTrainerConfig(root_dir='artifacts/model_trainer', data_path='artifacts/data_transformation/samsum_dataset', model_chkpt='google/pegasus-cnn_dailymail', num_train_epochs=1, warmup_steps=500, per_device_train_batch_size=1, per_device_eval_batch_size=1, weight_decay=0.01, logging_steps=10, evaluation_strategy='steps', eval_steps=500, save_steps=1e-06, gradient_accumulation_steps=16)


  6%|▌         | 3/51 [03:23<52:53, 66.11s/it]  

KeyboardInterrupt: 